In [ ]:
import pandas as pd
import os
import pickle

import networkx as nx

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from src.dataset.dataset_info import datasets
from src.graph.graph_measures import calculate_graph_measures
from src.graph.graph_construction.window_graph import create_weightless_window_graph 

In [ ]:
flow_graph = False
session_graph = False

window_graph = True
window_size=20000

multi_graph=False
line_graph = True

In [ ]:
# name = "cic_ids_2017_5_percent"
# name = "cic_ton_iot_5_percent"
# name = "cic_ton_iot"
name = "cic_ids_2017"
# name = "nf_bot_iot"
# name = "edge_iiot"
# name = "nf_cse_cic_ids2018"
# name = "nf_bot_iotv2"
# name = "nf_uq_nids"
# name = "x_iiot"
# name = "cic_ton_iot_modified"
# name = "nf_ton_iotv2_modified"
# name = "ccd_inid_modified"
# name = "nf_uq_nids_modified"

dataset = datasets[name]

In [ ]:
df = pd.read_parquet(dataset.path)

In [ ]:
df.head()

In [ ]:
cols_to_norm = list(set(list(df.columns ))  - set(list([dataset.label_col]))  - set(list([dataset.class_num_col])) - set(dataset.drop_columns)  - set(dataset.weak_columns))
cols_to_norm

In [ ]:
df.Label.value_counts()

In [ ]:
if window_graph and line_graph:
    df[dataset.timestamp_col] = pd.to_datetime(df[dataset.timestamp_col].str.strip(), format=dataset.timestamp_format)
    df.sort_values(dataset.timestamp_col, inplace=True)
    df.reset_index(drop=True, inplace=True)
    df.reset_index(drop=False, inplace=True)
    
    scaler = StandardScaler()

    df[cols_to_norm] = scaler.fit_transform(df[cols_to_norm])
    df['h'] = df[ cols_to_norm ].values.tolist()
    
    create_weightless_window_graph(
        df=df,
        src_ip_col=dataset.src_ip_col,
        dst_ip_col=dataset.dst_ip_col,
        window_size=window_size,
        line_graph=True,
        folder_path="datasets/" + name + "/line_graph",
        # folder_path=None,
        test_percentage = 20,
        edge_attr= ['h',dataset.label_col, "index"],
        file_type="pkl")
    
    

In [ ]:
if not window_graph:
        y = df[dataset.label_col]
        X_train, X_test, y_train, y_test = train_test_split(
                df, y, test_size=0.3, random_state=13, stratify=y)

        del df

        print(f"==>> X_train.shape: {X_train.shape}")
        print(f"==>> X_train.shape: {X_test.shape}")

        scaler = StandardScaler()

        X_train[cols_to_norm] = scaler.fit_transform(X_train[cols_to_norm])
        X_train['h'] = X_train[ cols_to_norm ].values.tolist()

        X_test[cols_to_norm] = scaler.transform(X_test[cols_to_norm])
        X_test['h'] = X_test[ cols_to_norm ].values.tolist()

In [ ]:
if flow_graph:
    G = nx.from_pandas_edgelist(X_train, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col], create_using=nx.MultiDiGraph())
    # get netowrk properties
    graph_measures = calculate_graph_measures(G, "datasets/" + name + "/training_graph_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    graph_measures = calculate_graph_measures(nx.DiGraph(G), "datasets/" + name + "/training_graph_simple_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    with open("datasets/" + name + "/training_graph.pkl", "wb") as f:
        pickle.dump(G, f)

In [ ]:
if flow_graph:
    G_test = nx.from_pandas_edgelist(X_test, dataset.src_ip_col, dataset.dst_ip_col, ['h',dataset.label_col],create_using=nx.MultiDiGraph())
    
    graph_measures = calculate_graph_measures(G_test, "datasets/" + name + "/testing_graph_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    graph_measures = calculate_graph_measures(nx.DiGraph(G_test), "datasets/" + name + "/testing_graph_simple_measures.json", verbose=True)
    print(f"==>> graph_measures: {graph_measures}")
    
    with open("datasets/" + name + "/testing_graph.pkl", "wb") as f:
        pickle.dump(G_test, f)

In [ ]:
# if window_graph and line_graph:

#     create_weightless_window_graph(
#         df=X_test,
#         src_ip_col=dataset.src_ip_col,
#         dst_ip_col=dataset.dst_ip_col,
#         window_size=window_size,
#         line_graph=True,
#         folder_path="datasets/" + name + "/line_graph/testing",
#         file_type="pkl")

In [ ]:
# nx.write_gexf(G_test, "datasets/cic_ton_iot/testing_graph.gexf")